# Eder Medina
## Homework 2

### APMTH 207: Stochastic Methods for Data Analysis, Inference and Optimization


## Problem 1: Monte Carlo Integration

Let $X$ be a random variable with distribution described by the following pdf:

$$
f_X(x) = \begin{cases}
\frac{1}{12}(x-1), &1\leq x\leq 3\\
-\frac{1}{12}(x-5), &3< x\leq 5\\
\frac{1}{6}(x-5), &5< x\leq 7\\
-\frac{1}{6}(x-9), &7< x\leq 9\\
0, &otherwise
\end{cases}
$$

Let $h$ be the following function of $X$:

$$
h(X) = \frac{1}{3\sqrt{2}\pi}\mathrm{exp}\left\{ -\frac{1}{18}\left( X - 5\right)^2\right\}
$$

Compute $\mathbb{E}[h(X)]$ via Monte Carlo simulation using the following sampling methods:
- inverse transform sampling
- rejection sampling with both uniform proposal distribution and normal proposal distribution (steroids) (with appropriately chosen parameters)
- importance sampling with both uniform proposal distribution and normal proposal distribution (with appropriately chosen parameters)

In [11]:
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [9]:
# PDF 
def  fx(x):
    if (x >= 1 and x <= 3):
        return 1./12*(x-1)
    elif(x > 3 and x <= 5):
        return -1./12*(x-5)
    elif(x > 5 and x <= 7):
        return 1/6*(x-5)
    elif(x > 7 and x <= 9):
        return -1./6*(x-9)
    else:
        return 0

# Function we are trying to estimate
def h(x):
    return 1.0/(3*np.sqrt(2)*np.pi)*np.exp(-1/18*(x-5)**2)



### Inverse Transform ###
We compute the CDF by integrating the PDF
$$
F_X(x) = \begin{cases}
0 , &x < 1 \\
\frac{1}{12}(\frac{x^2}{2}-x)+\frac{1}{24}, &1\leq x\leq 3\\
-\frac{1}{12}(\frac{x^2}{2}-5x)-\frac{17}{24}, &3< x\leq 5\\
\frac{1}{6}(\frac{x^2}{2}-5x)+\frac{29}{12}, &5< x\leq 7\\
-\frac{1}{6}(\frac{x^2}{2}-9x)-5.75, &7< x\leq 9\\
1 &x >9
\end{cases}
$$

We can compute the inverse cdf for each interval. For example, for the first interval we know that the $P(x<3)= 1/6$ then for values of $ r < 1/6$, where $r$ is defined by $P(x<X)= r$, we can compute the inverse transform by a quadratic equation
\begin{align}
r & = \frac{1}{12}(\frac{x^2}{2}-x)+\frac{1}{24}\\
& \frac{1}{12}(\frac{x^2}{2}-x)+\frac{1}{24} -r = 0 \\
x & = 1+\sqrt{1-4(\frac{1}{2})(\frac{1}{2}-12r)}
\end{align}

We then repeat this calculation for the other intervals.

In [107]:
# Define the CDF
def CDF(x):
    if (x >= 1 and x <= 3):
        return 1./12*(0.5*x*x-x)+1./24
    elif(x > 3 and x <= 5):
        return -1./12*(0.5*x*x-5*x)-17./24
    elif(x > 5 and x <= 7):
        return 1/6*(0.5*x*x-5*x)+29./12
    elif(x > 7 and x <= 9):
        return -1./6*(0.5*x*x-9*x)-5.75
    else:
        return 0

# Invert the CDF 
def invCDF(r):
    if (r<=1.0/6):
        return 1+np.sqrt(1-4*(1/2-12*r)*0.5)
    elif (r > 1.0/6 and r<= 1.0/3):
        return 5-np.sqrt(25-4*(17/2+12*r)*0.5)
    elif (r > 1.0/3 and r<= 2.0/3):
        return 5+np.sqrt(25-4*(29/2-6*r)*0.5)
    elif (r > 2.0/3 and r <= 1):
        return 9-np.sqrt(81-4*(6*5.75+6*r)*0.5)


We are then able to find
\begin{equation}
E(h(X)) = \frac{1}{N} \sum \limits_{x_i}^N h(x)
\end{equation}

## Problem 2: Variance Reduction

### Part A

Compute the variance of each estimate of $\mathbb{E}[h(X)]$ obtained in Problem 1. Based on the discussion on sampling methods in lecture, which sampling methods, proposal distributions is expected, in principle, to resulted in lower variances? How well do your results align with these expectations?

### Part B (Stratified Sampling)

Often, a complex integral can be computed with more ease if one can break up the domain of the integral into pieces and if on each piece of the domain the integral is simplified. 

- Find a natural way to divide the domain of $X$ and express $\mathbb{E}[h(X)]$ as an ***correctly*** weighted sum of integrals over the pieces of the domain of $X$. (This constitutes the essentials of Stratified Sampling)

- Estimate each integral in the summand using rejection sampling using a normal proposal distribution (with sensibly chosen parameters). From these, estimate $\mathbb{E}[h(X)]$.

- Compute the variance of your estimate of $\mathbb{E}[h(X)]$. Compare with the variance of your previous estimate of $\mathbb{E}[h(X)]$ (in Part A, using rejection sampling, a normal proposal distribution over the entire domain of $X$).

Read more about Stratified Sampling:

1. [Variance Reduction Techniques Slides](http://www.sta.nus.edu.sg/~zhangjt/teaching/ST4231/lectures/chapter4.pdf)

2. [Monte Carlo Methods](http://www.public.iastate.edu/~mervyn/stat580/Notes/s09mc.pdf)

3. [Variance Reduction Techniques Chapter](http://sas.uwaterloo.ca/~dlmcleis/s906/chapt4.pdf)